In [158]:
df = spark.read.csv('/Users/zhujinghong/Downloads/5003\ project/data/BRvideos.csv', header=True)

In [159]:
df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- publishedAt: string (nullable = true)
 |-- channelId: string (nullable = true)
 |-- channelTitle: string (nullable = true)
 |-- categoryId: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- view_count: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- description: string (nullable = true)



In [ ]:
# df = df.select('trending_date','title','channel_title',
#                'category_id', 'publish_time','tags','views',
#                'likes','dislikes','comment_count')

In [160]:
df = df.select('trending_date','title','channelTitle',
               'categoryId', 'publishedAt','tags','view_count',
               'likes','dislikes','comment_count')

In [163]:
df=df.withColumnRenamed('channelTitle','channel_title')
df=df.withColumnRenamed('categoryId','category_id') 
df=df.withColumnRenamed('publishedAt','publish_time')
df=df.withColumnRenamed('view_count','views')

In [164]:
df.printSchema()

root
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)



In [165]:
df.select('publish_time').show(truncate=False)

+--------------------+
|publish_time        |
+--------------------+
|2020-08-11T22:21:49Z|
|2020-08-11T15:00:13Z|
|2020-08-10T14:59:00Z|
|2020-08-11T15:00:09Z|
|2020-08-11T20:04:02Z|
|2020-08-11T17:14:20Z|
|2020-08-12T03:31:08Z|
|2020-08-11T00:02:35Z|
|2020-08-12T00:58:57Z|
|2020-08-12T08:02:01Z|
|2020-08-11T23:36:58Z|
|null                |
|null                |
|null                |
|null                |
|null                |
|null                |
|null                |
|null                |
|null                |
+--------------------+
only showing top 20 rows



In [166]:
# from pyspark.sql.functions import to_date, to_timestamp
# df.select(to_timestamp(df.publish_time, '%Y-%m-%dT%H:%M:%S.000Z').alias('dt')).show()

In [167]:
from pyspark.sql.functions import col , column
cols=['category_id','views','likes','dislikes','comment_count']
for col in cols:
    df=df.withColumn('%s'%col ,df[col].cast('int'))
    
df

DataFrame[trending_date: string, title: string, channel_title: string, category_id: int, publish_time: string, tags: string, views: int, likes: int, dislikes: int, comment_count: int]

In [168]:
df.select('views','likes','dislikes','comment_count').describe().show()

+-------+------------------+------------------+------------------+------------------+
|summary|             views|             likes|          dislikes|     comment_count|
+-------+------------------+------------------+------------------+------------------+
|  count|             91793|             91793|             91793|             91785|
|   mean|1698072.2923534475|125468.86622073578|2115.8671576263987|10693.848210491911|
| stddev| 6243165.547248336|427844.25914236804|11422.625312238624|109384.25383141158|
|    min|                 0|                 0|                 0|                 0|
|    max|         278080610|          16213756|            848980|           6817450|
+-------+------------------+------------------+------------------+------------------+



In [169]:
df.select('trending_date','title','channel_title','category_id', 'publish_time','tags').describe().show()

+-------+--------------------+--------------------+-----------------------+------------------+--------------------+----------------------+
|summary|       trending_date|               title|          channel_title|       category_id|        publish_time|                  tags|
+-------+--------------------+--------------------+-----------------------+------------------+--------------------+----------------------+
|  count|               92340|               95086|                  92859|             91793|               93624|                 92247|
|   mean|                null|                null|                   null|18.424890786879175|                null|                  null|
| stddev|                null|                null|                   null| 6.269286444084119|                null|                  null|
|    min|      Amanda Orestes|  sou a Anny tenh...|            Agnes Nunes|                 1|                    |                 2004)|
|    max|” and music video.

### remove null

In [170]:
from pyspark.sql.functions import isnull
# 查询某列为null的行数
cols = ['trending_date','title','channel_title',
        'category_id', 'publish_time','tags','views',
        'likes','dislikes','comment_count']
for col in cols:
    print(df1.filter(isnull('%s'%col)).count())

0
0
0
0
0
0
0
0
0
0


In [171]:
df.dropna(thresh=10).count()

91785

In [172]:
df1=df.dropna(thresh=10)
df1.count()

91785

### remove duplicate

In [173]:
# 1、删去完全重复的行
df2 = df1.dropDuplicates() 
df2.count()

91696

In [174]:
# 2.删除某些关键字段值完全相同的记录，subset参数定义这些字段
df2 = df2.dropDuplicates(subset = 
                         [c for c in df2.columns if c in ['channel_title','likes', 'dislikes','views','comment_count']])
df2.count()

91680

### outlier

In [126]:
cols = ['views', 'likes', 'dislikes','comment_count']
bounds = {}

for col in cols:
    quantiles = df2.approxQuantile(
        col, [0.25, 0.75], 0.05
    )
    
    IQR = quantiles[1] - quantiles[0]
    
    bounds[col] = [
        quantiles[0] - 1.5 * IQR,
        quantiles[1] + 1.5 * IQR
    ]
bounds

{'views': [-1737579.5, 3485976.5],
 'likes': [-56319.0, 107129.0],
 'dislikes': [-1931.5, 3712.5],
 'comment_count': [-5664.0, 10944.0]}

### json

In [175]:
df2.select('category_id','channel_title').show(truncate=False)

+-----------+---------------------+
|category_id|channel_title        |
+-----------+---------------------+
|26         |EntendendoiPhone     |
|17         |CONMEBOL Sudamericana|
|10         |MatheusKauanVEVO     |
|17         |UFC Brasil           |
|25         |TV BrasilGov         |
|24         |Felipe Neto          |
|17         |Jovem Pan Esportes   |
|17         |ESPN Brasil          |
|23         |Blogueirinha         |
|23         |André Fittipaldi     |
|10         |Josué Bom de Faixa   |
|24         |MTV                  |
|24         |Ramini Lima          |
|24         |O Victor Magalhães   |
|17         |TV Palmeiras/FAM     |
|23         |5 Alguma Coisa       |
|23         |Humor Multishow      |
|24         |Rádio Kiss FM Oficial|
|20         |Kamikat              |
|10         |CNCOVEVO             |
+-----------+---------------------+
only showing top 20 rows



In [176]:
j = spark.read.json('/Users/zhujinghong/Downloads/5003\ project/data/BR_category_id.json')

In [177]:
from pyspark.sql import functions as F
j = spark.read.option("multiLine","true").json('/Users/zhujinghong/Downloads/5003\ project/data/US_category_id.json')

# Explode Array to Structure
explodej = j.withColumn('Exp_RESULTS',F.explode(F.col('items'))).drop('items')

# Read location and name
dfj = explodej.select("Exp_RESULTS.snippet.title",'Exp_RESULTS.id')
dfj.show(truncate=False)

+---------------------+---+
|title                |id |
+---------------------+---+
|Film & Animation     |1  |
|Autos & Vehicles     |2  |
|Music                |10 |
|Pets & Animals       |15 |
|Sports               |17 |
|Short Movies         |18 |
|Travel & Events      |19 |
|Gaming               |20 |
|Videoblogging        |21 |
|People & Blogs       |22 |
|Comedy               |23 |
|Entertainment        |24 |
|News & Politics      |25 |
|Howto & Style        |26 |
|Education            |27 |
|Science & Technology |28 |
|Nonprofits & Activism|29 |
|Movies               |30 |
|Anime/Animation      |31 |
|Action/Adventure     |32 |
+---------------------+---+
only showing top 20 rows



In [178]:
dfj=dfj.withColumnRenamed('id','category_id')
dfj=dfj.withColumnRenamed('title','category_title')     # 将title重命名为category_title

In [179]:
dfj.show()

+--------------------+-----------+
|      category_title|category_id|
+--------------------+-----------+
|    Film & Animation|          1|
|    Autos & Vehicles|          2|
|               Music|         10|
|      Pets & Animals|         15|
|              Sports|         17|
|        Short Movies|         18|
|     Travel & Events|         19|
|              Gaming|         20|
|       Videoblogging|         21|
|      People & Blogs|         22|
|              Comedy|         23|
|       Entertainment|         24|
|     News & Politics|         25|
|       Howto & Style|         26|
|           Education|         27|
|Science & Technology|         28|
|Nonprofits & Acti...|         29|
|              Movies|         30|
|     Anime/Animation|         31|
|    Action/Adventure|         32|
+--------------------+-----------+
only showing top 20 rows



In [180]:
df3=df2.join(dfj, 'category_id', "left_outer")

In [181]:
df3.select('category_id','channel_title','title','category_title').show()

+-----------+--------------------+--------------------+--------------+
|category_id|       channel_title|               title|category_title|
+-----------+--------------------+--------------------+--------------+
|         27|  1a série EM - CMSP|09/02/21 - 1ª sér...|     Education|
|         24|         Babi Sabbag|COMO DESCOBRI MIN...| Entertainment|
|         24|Família Jeito lok...|O ENCONTRO DA MAY...| Entertainment|
|         23|      Mileninha Teen|MEU PAI ME TROLLO...|        Comedy|
|         17|          NBA Brasil|Los Angeles Laker...|        Sports|
|         17|          NBA Brasil|New York Knicks x...|        Sports|
|         24|       Tata Gulusian|O NOME DA MINHA F...| Entertainment|
|         10|Vitor Fernandes -...|             Aliança|         Music|
|         24|     Teodoro Sampaio|Live da Casa dos ...| Entertainment|
|         23|Humorista Thiago ...|FIZ TUDO COM ELA ...|        Comedy|
|         20|         NFA CHANNEL|FREE FIRE - NFA L...|        Gaming|
|     

In [182]:
df3.printSchema()

root
 |-- category_id: integer (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- category_title: string (nullable = true)



In [183]:
df3.coalesce(1).write.option("header", "true").csv("BR.csv")

In [184]:
df3.filter(isnull('likes')).count()

0